In [ ]:
import google_streetview
import google_streetview.api
import google_streetview.helpers
import os
import pandas as pd
import numpy as np
import datetime
import csv
import shutil

In [ ]:
# Convert data into dataframe of only longitude and latitude points that are novel
data = pd.read_csv('YOUR DATA.csv')
data_novel = data.loc[data['Novelty'] == 0]
data_novel.reset_index(drop=True, inplace=True)

print(data_novel)

In [ ]:
'''heading = 360
lat = 25.7126
long = -80.2803
APIkey = 'YOUR API KEY HERE'

params = [{
  'size': '640x640', # max 640x640 pixels
  'location': f"{lat}, {long}",
  'heading': str(heading),
  'pitch': '0',
  'key': APIkey
}]

# Create a results object
results = google_streetview.api.results(params)

# Download images to directory 'downloads'
results.download_links('downloads')'''

#Template for access of a google street view 640x640p given longitude, latitude, heading (0-360), and APIkey

In [ ]:
#Path for the main folder
file_path = r"\filepath\"
images_repository = f"{file_path}\Images"
logs_path = f"{file_path}\logs.csv"
print(os.path.exists(file_path))

In [ ]:
# Create Images / Metadata repository / logs csv - WILL NOT CREATE FOLDER if found in code
if os.path.exists(file_path):
    print(f"Error, must delete original folder {file_path}...")
else:
    # If it fails, inform the user.
    print(f"File not found, creating file {file_path}...")
    os.makedirs(file_path)
    os.makedirs(images_repository)

    # Create a new dataframe that will later become a logs.csv file
    with open(logs_path, 'w', encoding = 'UTF8') as csvfile:
        writer = csv.writer(csvfile)
        cols = ['lat', 'lon', 'found', 'file_name'] #header row
        writer.writerow(cols) #write header row to first row of logs

In [ ]:
APIkey = 'YOUR API KEY HERE' #My API Key

# Instantiate for loop that iterates through each row in data_novel, placing the respective images in folders.
for point in range(0, len(data_novel)):
    lat = data_novel.loc[point]["lat"] #Latitude comes from the 'lat' column
    long = data_novel.loc[point]["lon"] #Longitude comes from the 'lon' column
    date = data_novel.loc[point]["datevec"] #Date comes from the 'datevec' column
    subjvec = data_novel.loc[point]["subjvec"] #Subject ID from the 'subjvec' column
    filename = f"{images_repository}\{subjvec}_{date}_{lat}_{long}" #File names are in the format SUBJVEC_MM-DD-YYYY

    print(f"Loading... {point} of {len(data_novel)} completed!")

    
    params = {
        'size': '640x640', # max 640x640 pixels
        'location': f"{lat}, {long}", #For loop coordinates go here from each gps point instance in data_novel
        'heading': '0;90;180;270', #Four headings at 0, 90, 180, and 270 degrees for 360 degree vision
        'pitch': '0',
        'key': APIkey
    }


    # Check to see if the file path already exists - if so, add `_(whatever number it is on)`
    if os.path.exists(filename):
        for x in range (1, 1440):
            if not (os.path.exists(filename + f"_{x}")):
                filename = filename + f"_{x}"   
                break
    
    
    # See if this coordinate has been called before (be stingy on the # of API Calls) and if so end the iteration
    wasCalled = False

    with open(logs_path, 'r', encoding='UTF8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Iterate through each row in the CSV
        for row in reader:
            if ((str(lat) == row['lat']) and (str(long) == row['lon'])):
                if (str(row['found']) == '1'):
                    src = str(row['file_name'])
                    shutil.copytree(src, filename)
                    wasCalled = True
                    break
                elif (str(row['found']) == '0'):
                    wasCalled = True
                    break
                else:
                    print("Weird...")

    
    if (wasCalled):
        continue
    

    # Get a list of all possible queries from multiple parameters
    api_list = google_streetview.helpers.api_list(params)
    
    # Create a results object for all possible queries
    results = google_streetview.api.results(api_list)

    
    # If the image is found (it is not NOT_FOUND, or returns ZERO_RESULTS): save metadata to metadata, download file, and write instance to logs.csv

    if ((str(results.metadata[0]['status']) != "NOT_FOUND") and (str(results.metadata[0]['status']) != "ZERO_RESULTS")):
        
        # Add this point so this lat long never has to be called again - just copied from another folder
        with open(logs_path, 'a', encoding = 'UTF8') as csvfile:
            writer = csv.writer(csvfile)
            cols = [lat, long, 1, filename] #header row
            writer.writerow(cols) #write header row to first row of logs
        
        results.download_links(filename, metadata_file = r"\file_location\")
        
    else:
        # Same thing but for points with no images found
        print("NOT FOUND")
        with open(logs_path, 'a', encoding = 'UTF8') as csvfile:
            writer = csv.writer(csvfile)
            cols = [lat, long, 0, filename] #header row
            writer.writerow(cols) #write header row to first row of logs



print(f"Loading... {len(data_novel)} of {len(data_novel)} completed!")